# API Nakala

## Packages

In [14]:
using CSV
using DataFrames
using HTTP
using JSON
using Dates

## Identifiants

In [21]:
path = @__DIR__
credentials = CSV.read(joinpath(path,"credentials/credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

"01234567-89ab-cdef-0123-456789abcdef"

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [22]:
apitest = true

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

"https://apitest.nakala.fr"

## Création d'une collection

Une fois les identifiants chargés et l'API sélectionnée, entrez simplement un nom de collection pour la créer.

In [23]:
collectionName = "V1"

url = joinpath(apiurl, "collections")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :status => "private",
  :metas =>  [Dict(
    :value => collectionName,
    :propertyUri => "http://nakala.fr/terms#title",
    :typeUri => "http://www.w3.org/2001/XMLSchema#string",
    :lang => "fr"
  )]
)

postCollection = HTTP.request("POST", url, headers, JSON.json(body)) # envoi des données pour la création de la collection
collectionResponse = JSON.parse(String(HTTP.payload(postCollection))) # réponse du server
collectionId = collectionResponse["payload"]["id"] # récupération de l'id de la collection
print("Identifiant collection : ", collectionId)

Identifiant collection : 10.34847/nkl.3edb5stu

## Informations utilisateur

In [24]:
url = joinpath(apiurl, "users", "me")

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

userInfo = HTTP.request("GET", url, headers)
userInfoResponse = JSON.parse(String(HTTP.payload(userInfo))) # réponse du server
username = userInfoResponse["username"]
userGroupId = userInfoResponse["userGroupId"]
userInfoResponse

Dict{String, Any} with 11 entries:
  "firstLogin"  => "2020-03-18T11:20:00+01:00"
  "username"    => "tnakala"
  "surname"     => "Nakala"
  "userGroupId" => "26cef362-5bef-11eb-99d1-5254000a365d"
  "lastLogin"   => "2024-01-19T18:24:43+01:00"
  "mail"        => "nakala@huma-num.fr"
  "photo"       => "b\\xcf\\x84o\\xcf\\x81\\xce\\xbdo\\xcf\\x82"
  "fullname"    => "Test Nakala"
  "givenname"   => "Test"
  "apiKey"      => "01234567-89ab-cdef-0123-456789abcdef"
  "roles"       => Any["ROLE_USER"]

## Ressources utilisateur

In [25]:
scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]

url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)
userDatas = HTTP.request("POST", url, headers)
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

Dict{String, Any} with 2 entries:
  "totalRecords" => 1240
  "data"         => Any[Dict{String, Any}("isDepositor"=>true, "isOwner"=>true,…